# E-06 프로젝트 : 네이버 영화리뷰 분석

## 시작하기 전에

실습 단계에서 수많은 시행착오를 거치며 프로젝트는 시작도 하지 못하고 제출일에 도착했다.

헤매는 과정에서 본의 아니게 많은 공부를 하게 되어 후회는 없지만 한 노트북에 작성하기에는

그 지난한 과정 중에, 자잘한 수정을 거치는 반복 작업이 너무 많다.

가독성을 위해 프로젝트 노트를 따로 작성하되, 학습 당한(..)내용들을 중간중간 삽입할 것.

## 0. 사전준비

In [1]:
!pip install jupyterthemes
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme
from jupyterthemes import jtplot

In [2]:
set_nb_theme('onedork')

In [3]:
import glob
import os
import re
import numpy as np
import pandas as pd

from collections import Counter

import urllib.request
from konlpy.tag import Okt, Mecab

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import Constant

from gensim.models import KeyedVectors
import gensim
from sklearn.model_selection import train_test_split

from keras.layers import Bidirectional, LSTM, GRU, Dense, Conv1D
from keras.layers import Embedding, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
!pwd

/aiffel/aiffel/Exploration/film_is_naver


In [5]:
path = '/aiffel/aiffel/Exploration/film_is_naver/'

In [6]:
# 네이버 영화 댓글 코퍼스를 링크
!ln -s ~/data/*.txt /aiffel/aiffel/Exploration/film_is_naver/data

ln: failed to create symbolic link '/aiffel/aiffel/Exploration/film_is_naver/data/ratings_test.txt': File exists
ln: failed to create symbolic link '/aiffel/aiffel/Exploration/film_is_naver/data/ratings_train.txt': File exists


## 1. 데이터 확인, 전처리

### 1.1. 데이터 준비 및 확인

In [7]:
import pandas as pd

# 데이터를 읽어봅시다. 
train_data = pd.read_table(path+'data/ratings_train.txt')
test_data = pd.read_table(path+'data/ratings_test.txt')

print(train_data.shape)
print(test_data.shape)

print(train_data.columns)

train_data.head()


(150000, 3)
(50000, 3)
Index(['id', 'document', 'label'], dtype='object')


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 1.2. 데이터로더 구성
- 데이터의 중복 제거
- NaN 결측치 제거
- 한국어 토크나이저로 토큰화
- 불용어(Stopwords) 제거
- 사전word_to_index 구성
- 텍스트 스트링을 사전 인덱스 스트링으로 변환
- X_train, y_train, X_test, y_test, word_to_index 리턴

위 항목을 구현하기 위한 코드가 아래와 같이 제공되어있다.

In [8]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# def load_data(train_data, test_data, num_words=num_words):
#     # [[YOUR CODE]]
    
# X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

konlpy.tag
> 파이썬 한국어 NLP, KoNLPy(코엔엘파이), 한국어 정보처리를 위한 파이썬 패키지.

Mecab
> konlpy의 서브패키지(dtype=class), 형태소 분석을 위한 패키지.

형태소
>1. 의미를 가지는 요소로서는 더 이상 분석할 수 없는 가장 작은 말의 단위
>2. 문법적·관계적인 뜻만을 나타내는 단어 또는 단어의 부분.

불용어
> 데이터에서 유의미한 단어 토큰을 선별하기 위해서, 큰 의미가 없는 단어 토큰을 제거하는 작업

In [9]:
# 얼마간 직접 코드 작성을 시도해보았다. 
# 예시 답안의 코드 길이를 보고 빠른 항복을 선언했다.(이미 너무 많은 시간이 학습에서 지체되었다..)
# 주석으로 간단히 리뷰만 하도록 하자.

def load_data(train_data, test_data, num_words=10000):
    # 문장 부분의 중복을 제거한다. 완전히 동일한 내용의 리뷰(도배..?)를 여러 번 학습 할 이유는 없다.
    train_data.drop_duplicates(subset=['document'], inplace=True)
    # 결측치가 하나라도 있는 항목은 제거한다.
    train_data = train_data.dropna(how = 'any')
    # 위와 동일
    test_data.drop_duplicates(subset=['document'], inplace=True)
    test_data = test_data.dropna(how = 'any') 
    
    #학습용 데이터를 담을 리스트
    X_train = []
    # 데이터중 코퍼스에 해당하는 컬럼을 토큰화 한다. 
    # 토큰화 메소드는 morphs()로 '구' 단위를 형태소 단위로 파싱한다.
    # 한국어의 특성 상 어절 단위로 토큰화 하면 안 되니, '구'를 대상으로 하는 듯. 후에 조금 더 살펴볼 예정.
    # 토큰화 된 리스트에서 불용어를 제거하는 내포for문.
    # 토큰화와 불용어 제거를 거친 문장들을 위의 빈 리스트에 담는다.
    for sentence in train_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_train.append(temp_X)

    # 위와 동일
    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_test.append(temp_X)
    
    # 넘파이의 concatenate메소드로 전체 토큰을 한 줄짜리 배열로 만든다. 그리고 다시 리스트로 반환
    words = np.concatenate(X_train).tolist()
    # 콜렉션 모듈의 카운터 클래스는 hashable한* 객체를 인자로 받아, 동일한 값의 갯수를 밸류로 하는 dict로 반환한다. 
    # 즉 words변수에 담긴 모든 서로다른 토큰의 갯수를 세어주는 것.
    # hashable에 대해 아래 마크다운에 좋은 포스팅을 링크
    counter = Counter(words)
    # most_commmon메소드를 이용하면 가장 갯수가 많은 순으로 정렬된 튜플을 리턴한다.
    # 정수 인자를 주면, 가장 갯수가 많은 값 부터 해당 정수개를 반환.
    # 즉 가장 고빈도로 사용된 10000-4 개의 토큰을 갯수와 함께 반환할 것이다.
    # 10000-4는 추후 추가될 pad, bos, unk와 paddind 처리를 고려한 숫자 같다.
    counter = counter.most_common(10000-4)
    # 단어장 리스트의 앞쪽 네 자리에 공백을 주기 위한 리스트에 
    # for내포 리스트를 생성한다. for문에서 언더바는 값을 무시할 때 쓰인다고 한다.
    # 따라서 네 자리 공백 + 빈도순 토큰으로 이뤄진 리스트가 생성된다.
    vocab = ['', '', '', ''] + [key for key, _ in counter]
    # 이 리스트를 다시 순서대로 인덱싱해 dict로 저장.
    word_to_index = {word:index for index, word in enumerate(vocab)}
    
    # wordlist를 받아, 단어가 토큰 리스트에 있으면 인덱스를 반환하고, 그렇지 않다면 공백에 대한 인덱스를 반환.
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index[''] for word in wordlist]
    
    # 데이터셋의 문장들을 단어장과 맵핑한다. 빈도수 밖의 단어들이 포함되어 있다면 공백에 대한 인덱스로 바뀔 것.
    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))
    
    # 결과로 정제된 토큰데이터셋과 라벨의 배열을 반환한다.
    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index

hashable이란?
> https://analytics4everything.tistory.com/138

In [10]:
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

In [11]:
word_to_index['<PAD>']=0
word_to_index['<BOS>']=1
word_to_index['<UNK>']=2
word_to_index['<UNUSED>'] = 3
index_to_word = {index:word for word, index in word_to_index.items()}

In [12]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

### 1.3. 데이터 분석 및 가공
- 데이터셋 내 문장 길이 분포
- 적절한 최대 문장 길이 지정
- keras.preprocessing.sequence.pad_sequences 을 활용한 패딩 추가


In [13]:
 get_decoded_sentence(X_train[4], index_to_word)

'익살 스런 연기 돋보였 던 영화 ! 스파이더맨 에서 늙 어 보이 기 만 했 던 <UNUSED> <UNUSED> 너무나 이뻐 보였 다'

In [14]:
len(X_train)

146182

In [15]:
total_data_text = list(X_train) + list(X_test)

num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 최대 길이를 (평균 + 3*표준편차)로 설정. 97%를 포함한다.
# 문장의 평균 길이는 약 16, 표준편차는 약 13이다.
# 앞서 학습한 내용을 토대로, padding 설정은 pre로 할 것이기 때문에 평균에서 조금 많이 벗어나도 무관할 것 같다.
max_tokens = np.mean(num_tokens) + 3 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  15.96940191154864
문장길이 최대 :  116
문장길이 표준편차 :  12.843571191092
pad_sequences maxlen :  54
전체 문장의 0.9720946661956905%가 maxlen 설정값 이내에 포함됩니다. 


### padding... 그리고 model의 가중치
- 지난 작사가 인공지능 exp에 이어 이번에도 padding이 난항의 시작점이었다.
- pre 하이퍼파라미터가 더 적합하다는 학습내용의 설명은 쉽게 확인되었다.
- 하지만 그 과정에서 post와 pre 모두를 확인했으며, epoch을 바꿔가며 확인해보라는 부분까지 수행하며 문제가 생겼다.


        #원본데이터를 불러오고, 패딩을 각각 pre와 post로 처리해주고, 밸리데이션 데이터를 스플릿해주는 과정을 함수로 지정
        def data_init_split(x_train, x_test):
        x_train_pre = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                                value=word_to_index["<PAD>"],
                                                                # padding='post',
                                                                maxlen=maxlen)
    
        x_test_pre = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                               value=word_to_index["<PAD>"],
                                                               # padding='post',
                                                               maxlen=maxlen)
    
        x_train_post = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                                value=word_to_index["<PAD>"],
                                                                padding='post',
                                                                maxlen=maxlen)
    
        x_test_post = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                               value=word_to_index["<PAD>"],
                                                               padding='post',
                                                               maxlen=maxlen)
        # validation set 10000건 분리, 패딩 pre와 post를 각각생성
        
        x_val_pre = x_train_pre[:10000]   
        x_val_post = x_train_post[:10000]   
        y_val = y_train[:10000]
        
        # validation set을 제외한 나머지 15000건
        x_train_pre = x_train_pre[10000:]  
        x_train_post = x_train_post[10000:]  
        partial_y_train = y_train[10000:]
        
        print(type(x_train_pre))
        print(x_train_pre.shape, x_train_post.shape, partial_y_train.shape)
        return(x_train_pre, x_test_pre, x_train_post, x_test_post, x_val_pre, x_val_post, partial_y_train, y_val)
        
- 이렇게 나누어 여러 번 학습을 거듭하는 과정에서 첫 epoch부터 과적합이 일어난 상태로 학습이 진행된 것.

- 사실 함수까지 지정한 이유도 학습이 데이터셋 자체에 영향을 주는가 하는 말도 안되는 의심에서부터였다.

- 긴 헛발질이 시작된 이유는...
    - 이전까지의 exp에서 여러 레이어를 가진 모델을, 즉 딥러닝 모델을 사용한 적이 없지는 않았다.
    - 그러나 이전까지는 모델의 생성이 함수로 지정되어있거나.. 등등 모델이 재정의 된 상태였다.
    - 그래서 동일한 셀에서 여러번 fit시키는 것이 계속해서 업데이트 된 가중치를 반영할 것이라고 상상치 못했다.
    > - 어마어마한 헛발질로 시간을 소비했다.
    

원인을 파악하기 위한 헛발질 추론과 그에 따라 학습당한(..) 내용은 대략 아래와 같다.

- trainable=True  # trainable을 True로 주면 Fine-tuning
> 이 한 줄의 설명은 제법 그럴듯 한 이유로 보였다. 결과적으로는 무관했으나 레이어의 동결을 조절하는 방식에 대해 수차례 학습하게 되었다.


- 임베딩레이어의 특성인가?
> 임베딩 레이어의 초기화 파라미터는 다른 레이어와 조금 다른 면이 있었다. 사전학습 된 임베딩 레이어를 카피하는 과정에서 이니셜라이저=Constant 부분이 마음에 걸려 한참을 찾아보았다. 하지만 fit이 포함된 셀을 실행할 때마다 과적합은 발생했지만, 임베딩 레이어의 가중치는 변화하지 않았다. 그렇다면 다른 레이어에서 가중치가 변한다는 것인데 아무리 봐도 특이한 점은 없어 보였다.


- 모델이 데이터셋에 무언가 영향을 주고 있는 것은 아닐까?
- 컴파일 과정에서 무언가 빠진 것은 아닐까?
- history 변수에 담는 것이 문제인가?
- 시퀀셜 모델의 특수성인가?
- mask_zero=True 파라미터 때문인가?
- 혹시 keras 버전 차이에서 발생하는 문제는 아닌가
> 등등 온갖 뻘짖을 하며 keras 공식문서를 뒤적였음..

~~결론적으로 그냥 그게 정상이었다.~~

### 딥러닝.모델의.레이어.
생각해 보면 당연한 일이다. 모델의 학습이 반복되며 앞서 학습한 내용을 반영해 가중치가 변하지 않는다면 애초에 딥러닝은 성립도 하지 않을 것이다. 

하지만 이 상황에 대해 검색을 해 보아도, 질문을 했을 때에도 명쾌한 대답을 듣지 못한 덕분에 많은 공부를 했다(..)

결론적으로 실습 과정에서 여러 번 동일한 모델로 학습을 시켜야 할 경우
- 모델과 레이어를 구성하는 과정을 함수로 지정하면 끝.

일정 수준까지 파인튜닝되었거나, 학습이 진행된 상태에서 다른 데이터셋에 적용시켜보고 싶은 경우
- 그 상태의 모델을 딥카피해서 사용하면 끝.

이미 너무 길었지만 각설하고 다시 진행해보자
### 패딩추가

In [16]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                        value=word_to_index["<PAD>"],
                                                        # padding='post', 디폴트가 pre.
                                                        maxlen=maxlen)
x_test = tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                       value=word_to_index["<PAD>"],
                                                       maxlen=maxlen)

print(x_train.shape)
print(x_test.shape)

(146182, 54)
(49157, 54)


#### 실습보다 훨씬 많은 양의 데이터셋이므로 분할 비율을 달리 해야할 것
비율은 3:1 정도로 하자

In [17]:
x_val = x_train[:len(X_train)//4]   
y_val = y_train[:len(y_train)//4]

partial_x_train = x_train[len(X_train)//4:]  
partial_y_train = y_train[len(y_train)//4:]

print(partial_x_train.shape)
print(partial_y_train.shape)

(109637, 54)
(109637,)


In [62]:
# 최초 함수 지정 때 부터 염두에 둔 단어 사전의 수
vocab_size = 10000
# 실습 단계보다 학습량이 훨씬 늘었으니 약간 늘려보자
word_vector_dim = 8

## 2. 모델 구성 및 학습

### 2.1. 1-D CNN

In [63]:
# 1-D CNN 모델 구성
def CNN_1D():
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
    model.add(keras.layers.Conv1D(32, 3, activation='relu'))
    model.add(keras.layers.MaxPooling1D(5))
    model.add(keras.layers.Conv1D(16, 3, activation='relu'))
    model.add(keras.layers.GlobalMaxPooling1D())
    model.add(keras.layers.Dense(8, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim
    
    model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
    
    return model

In [67]:
model = CNN_1D()
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 8)           80000     
_________________________________________________________________
conv1d_18 (Conv1D)           (None, None, 32)          800       
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, None, 16)          1552      
_________________________________________________________________
global_max_pooling1d_9 (Glob (None, 16)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                

In [68]:
#학습
history4 = model.fit(partial_x_train,
                   partial_y_train,
                   epochs=5,
                   batch_size=64,
                   validation_data=(x_val, y_val),
                   verbose=1)

Epoch 1/5
1714/1714 [==============================] - 8s 4ms/step - loss: 0.4432 - accuracy: 0.7833 - val_loss: 0.3822 - val_accuracy: 0.8224
Epoch 2/5
1714/1714 [==============================] - 7s 4ms/step - loss: 0.3463 - accuracy: 0.8446 - val_loss: 0.3816 - val_accuracy: 0.8236
Epoch 3/5
1714/1714 [==============================] - 8s 5ms/step - loss: 0.3062 - accuracy: 0.8656 - val_loss: 0.3890 - val_accuracy: 0.8250
Epoch 4/5
1714/1714 [==============================] - 7s 4ms/step - loss: 0.2738 - accuracy: 0.8825 - val_loss: 0.4101 - val_accuracy: 0.8198
Epoch 5/5
1714/1714 [==============================] - 7s 4ms/step - loss: 0.2453 - accuracy: 0.8964 - val_loss: 0.4332 - val_accuracy: 0.8172


In [69]:
#평가
results = model.evaluate(x_test, y_test, verbose=2)

print(results)

1537/1537 - 2s - loss: 0.4398 - accuracy: 0.8141
[0.4397837817668915, 0.8140854835510254]


In [49]:
set1 = print('1회차',
             'word_vector_dim = 4',
             'epoch = 5',
             'batch_size=256',
              results, sep='\n')


history.history

1회차
word_vector_dim = 4
epoch = 5
batch_size=256
[0.40767908096313477, 0.8149195313453674]


{'loss': [0.5065277814865112,
  0.3780733644962311,
  0.3520091772079468,
  0.33496713638305664,
  0.3196852207183838],
 'accuracy': [0.74005126953125,
  0.8297746181488037,
  0.8435838222503662,
  0.8522396683692932,
  0.8612329959869385],
 'val_loss': [0.40680819749832153,
  0.39555567502975464,
  0.39175570011138916,
  0.3945945203304291,
  0.3998227119445801],
 'val_accuracy': [0.811246395111084,
  0.8162813186645508,
  0.8181146383285522,
  0.817895770072937,
  0.8179778456687927]}

In [55]:
set2 = print('2회차',
             'word_vector_dim = 8',
             'epoch = 5',
             'batch_size=256',
              results, sep='\n')

history2.history

2회차
word_vector_dim = 8
epoch = 5
batch_size=256
[0.4143110513687134, 0.8156518936157227]


{'loss': [0.4904283583164215,
  0.36823388934135437,
  0.3362371027469635,
  0.31119629740715027,
  0.28670284152030945],
 'accuracy': [0.7528206706047058,
  0.8347911834716797,
  0.8515099883079529,
  0.8649634718894958,
  0.8780521154403687],
 'val_loss': [0.39699262380599976,
  0.3880945146083832,
  0.3856084942817688,
  0.3925116956233978,
  0.4080141484737396],
 'val_accuracy': [0.8122588396072388,
  0.8206594586372375,
  0.821042537689209,
  0.8220276236534119,
  0.819346010684967]}

In [61]:
set3 = print('3회차',
             'word_vector_dim = 16',
             'epoch = 5',
             'batch_size=256',
              results, sep='\n')

history3.history

3회차
word_vector_dim = 16
epoch = 5
batch_size=256
[0.4420379102230072, 0.8156926035881042]


{'loss': [0.48505234718322754,
  0.3563792407512665,
  0.3166244626045227,
  0.2816544473171234,
  0.2456458956003189],
 'accuracy': [0.7564325928688049,
  0.8405009508132935,
  0.8619717955589294,
  0.8796391487121582,
  0.8975072503089905],
 'val_loss': [0.3914943039417267,
  0.38320499658584595,
  0.38673651218414307,
  0.4023144543170929,
  0.4359022378921509],
 'val_accuracy': [0.8179778456687927,
  0.8238336443901062,
  0.8243535161018372,
  0.8221371173858643,
  0.8170201182365417]}

#### 워드벡터의 차원 수 조절
- epoch이 적고, 과적합이 빨리 일어나는 편이라 큰 영향이 없어 보이지만 
<br>4 8 16에서 각각 val_loss가 상승하기 시작한 시점이 다르다.

In [75]:
set4 = print('4회차',
             'word_vector_dim = 16',
             'epoch = 5',
             'batch_size=64',
              results, sep='\n')

history4.history

4회차
word_vector_dim = 16
epoch = 5
batch_size=64
[0.4397837817668915, 0.8140854835510254]


{'loss': [0.4432016611099243,
  0.3462603688240051,
  0.30615949630737305,
  0.2737511694431305,
  0.24530825018882751],
 'accuracy': [0.7833030819892883,
  0.8446145057678223,
  0.8655837178230286,
  0.8825305104255676,
  0.8963853716850281],
 'val_loss': [0.38220542669296265,
  0.38160228729248047,
  0.3890046775341034,
  0.41014906764030457,
  0.4331640899181366],
 'val_accuracy': [0.8224380612373352,
  0.8236420750617981,
  0.8250375986099243,
  0.8197838068008423,
  0.8171569108963013]}

#### 배치사이즈
- 를 줄여보았더니 더 빠르게 과적합이 일어났다. 
- 1D-CNN 특성상 가벼운 하이퍼파라미터 설정은 큰 영향을 끼치지 못할 것 같다는 생각이 든다.
- 결과는 대부분 비슷했다
> 최적 val_loss : 약 0.38
>
> 정확도 평가점수 : 약 0.81 

### 2.2. GlobalMaxPooling1D

In [72]:
# 모델 구성
def GMP_1D():
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
    model.add(keras.layers.GlobalMaxPooling1D())
    model.add(keras.layers.Dense(8, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
    
    return model

In [79]:
# 학습
word_vector_dim = 12
model = CNN_1D()
history = model.fit(partial_x_train,
                   partial_y_train,
                   epochs=5,
                   batch_size=256,
                   validation_data=(x_val, y_val),
                   verbose=1)

Epoch 1/5
429/429 [==============================] - 3s 5ms/step - loss: 0.4956 - accuracy: 0.7511 - val_loss: 0.4028 - val_accuracy: 0.8098
Epoch 2/5
429/429 [==============================] - 2s 5ms/step - loss: 0.3637 - accuracy: 0.8374 - val_loss: 0.3870 - val_accuracy: 0.8209
Epoch 3/5
429/429 [==============================] - 2s 5ms/step - loss: 0.3294 - accuracy: 0.8560 - val_loss: 0.3891 - val_accuracy: 0.8235
Epoch 4/5
429/429 [==============================] - 2s 5ms/step - loss: 0.3020 - accuracy: 0.8704 - val_loss: 0.4026 - val_accuracy: 0.8197
Epoch 5/5
429/429 [==============================] - 2s 5ms/step - loss: 0.2756 - accuracy: 0.8838 - val_loss: 0.4121 - val_accuracy: 0.8200


In [76]:
results = model.evaluate(x_test, y_test, verbose=2)

print(results)

1537/1537 - 2s - loss: 0.4353 - accuracy: 0.8151
[0.43527817726135254, 0.815082311630249]


#### gmp1d 요약
- 1D-CNN과 큰 차이는 없었다.(2-4epoch 부터 과적합 발생, loss, 정확도 비슷)
- 이번에도 워드백터차원수, 배치사이즈 정도의 파라미터 조절로는 큰 변화는 없었다.
- 역시 레이어의 깊이가 얕기 때문이 아닐까 생각됨.
> 최적 val_loss : 약 0.38
>
> 정확도 평가점수 : 약 0.815 

### 2.3. LSTM 레이어

In [80]:
# 모델 구성
def LSTM():
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
    model.add(keras.layers.LSTM(8))  
    model.add(keras.layers.Dense(8, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))  

    model.summary()

    model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
    
    return model

In [85]:
# 학습
word_vector_dim = 16
model = LSTM()
history = model.fit(partial_x_train,
                   partial_y_train,
                   epochs=7,
                   batch_size=256,
                   validation_data=(x_val, y_val),
                   verbose=1)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, None, 16)          160000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 8)                 800       
_________________________________________________________________
dense_36 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 9         
Total params: 160,881
Trainable params: 160,881
Non-trainable params: 0
_________________________________________________________________
Epoch 1/7
429/429 [==============================] - 4s 7ms/step - loss: 0.4579 - accuracy: 0.7970 - val_loss: 0.3571 - val_accuracy: 0.8458
Epoch 2/7
429/429 [==============================] - 3s 6ms/step - loss: 0.3365 - accuracy: 0.8573 - val

In [86]:
results = model.evaluate(x_test, y_test, verbose=2)

print(results)

1537/1537 - 3s - loss: 0.3614 - accuracy: 0.8439
[0.36141589283943176, 0.8438879251480103]


#### LSTM 요약
- 기본적으로 앞서 모델들 보다 결과가 좋았음.
- 과적합이 일어나는 시기가 덜 빨리 찾아옴
- 워드벡터의 차원수를 4 8 16으로 비교해보았을 때 높은 차원에서 조금 더 균일한 학습 지표를 보여줌
> 최적 val_loss : 약 0.345
>
> 정확도 평가점수 : 약 0.845 

## 3. Embedding Layer 분석

In [87]:
from gensim.models.keyedvectors import Word2VecKeyedVectors

In [88]:
embedding_layer = model.layers[0]
weights = embedding_layer.get_weights()[0]
print(weights.shape) 

(10000, 16)


In [91]:
path

'/aiffel/aiffel/Exploration/film_is_naver/'

In [92]:
word2vec_file_path = path + '/data/word2vec.txt'
f =  open(word2vec_file_path, 'w')
f.write('{} {}\n'.format(vocab_size-4, word_vector_dim))

vectors = model.get_weights()[0]
for i in range(4,vocab_size):
    f.write('{} {}\n'.format(index_to_word[i], ' '.join(map(str, list(vectors[i, :])))))
f.close()

In [93]:
word_vectors = Word2VecKeyedVectors.load_word2vec_format(word2vec_file_path, binary=False)

In [94]:
word_vectors.similar_by_word("별로")

[('22', 0.9826436042785645),
 ('재미없', 0.9786253571510315),
 ('억지', 0.978186309337616),
 ('OOOO', 0.9777253866195679),
 ('어설픈', 0.9729003310203552),
 ('대충', 0.9710397124290466),
 ('미화', 0.9705512523651123),
 ('천박', 0.9699770212173462),
 ('역겨움', 0.9682828187942505),
 ('친일파', 0.9675014615058899)]

#### 생각보다 너무 잘 하는데?

### 3.1.한국어 버전 Word2Vec Embedding

In [95]:
gensim.__version__

'4.1.2'

In [96]:
!pip install --upgrade gensim==3.8.3

     |████████████████████████████████| 23.4 MB 6.5 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-linux_x86_64.whl size=24328218 sha256=e3955eacafc6a718155fb922dd3f8a008270c96fe585271bffec2e461bc34cb8
  Stored in directory: /aiffel/.cache/pip/wheels/ca/5d/af/618594ec2f28608c1d6ee7d2b7e95a3e9b06551e3b80a491d6
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.1.2
    Uninstalling gensim-4.1.2:
      Successfully uninstalled gensim-4.1.2


In [97]:
gensim.__version__

'4.1.2'

### 다운그레이드 이슈
- LMS 노트북에서는 pip를 사용하는 것이 어렵다.. 추후해결하자

In [99]:
word2vec_path = path + 'data/ko.bin'
K_model = gensim.models.Word2Vec.load(word2vec_path)

AttributeError: Can't get attribute 'Vocab' on <module 'gensim.models.word2vec' from '/opt/conda/lib/python3.9/site-packages/gensim/models/word2vec.py'>

## 급마무리

### 기억에 남는 학습 내용
- 온갖 keras 공식문서.


### 어려웠던 점
- .. 신경망 모델에 대한 이해 부족.


### 추가로 해보고 싶은 점
- exp8 역시 자연어와 관련된 것이니 그 때는 기필코 그 부분에 중점을 두고 학습할 것이다.
- gensim 다운그레이드를 진행하지 못해 시간상 못 한 한국어임베딩 레이어의 전이학습은 따로 시도해보겠습니다.


### 총평
: 아.. 이번 연휴를 불태워 사이클을 정상화시키자.